In [136]:
import seaborn as sns
import numpy as np
import pandas as pd
import statistics as st 
import matplotlib.pyplot as plt
import sklearn.model_selection as sk 
df=pd.read_csv(r"C:\Users\Sara\Downloads\titanic-Dataset.csv")
#understand data
print(df.describe())
print(df.info())
#missing values in Age
#missing values in Cabin
#mssing value in Embarked
#fill the missing ones with the mode
l=["Age","Cabin","Embarked"]
for i in l:
    if i=="Age":
        x=int(st.mean(list(df[i].loc[df[i].isna()==False])))
    else:
        x=st.mode(list(df[i].loc[(df[i].isna()==False)]))
    df[i].fillna(x,inplace=True)
#visualize each column
n_df=df.loc[:, ~df.columns.isin(['PassengerId', 'Name','Sex',"Cabin","Embarked","Ticket"])]
for i in list(n_df.columns):
    print(i)
    plt.boxplot(n_df[i])
    plt.show()
#Age contains outliers, they are acceptable
#SibSp contains outliers, they are not acceptable
#Parch contains outliers, they are acceptable
#Fare contains outliers, they are not acceptable
#Remove outliers from Fare and SibSp 
l1=["Fare","SibSp"]
def R(A,name):
    q1 = np.quantile(list(A[name]), 0.10)
    q3 = np.quantile(list(A[name]), 0.90)
    med = np.median(list(A[name]))
    iqr = q3-q1
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    A1=list(A.loc[(A[name]<upper_bound)&(A[name]>lower_bound)][name])
    A.loc[A[name]>upper_bound]=int(st.mean(A1))
    A.loc[A[name]<lower_bound]=int(st.mean(A1))
    return A[name]
for i in l1:
    df[i]=R(df,i)
n_df=df.loc[:, ~df.columns.isin(['PassengerId', 'Name','Sex',"Cabin","Embarked","Ticket"])]
def robust_Scaling(p):
    lis=list(p)
    q1 = np.quantile(lis, 0.10)
    q3 = np.quantile(lis, 0.90)
    med = np.median(lis)
    iqr = q3-q1
    p=(p-med)/iqr
    return list(p)
c=list(n_df.columns)
for i in c:
    df[i]=robust_Scaling(df[i])
sns.pairplot(n_df)
#Encoding
#k-fold:Name, Cabin, Ticket
#label:Sex,
#One-hot:Embarked
def k_fold_category_stats(data, target_col, category_col, k):
    kf = sk.KFold(n_splits=k, shuffle=True)
    
    # Copy the dataframe to avoid changing the original data
    data_copy = data.copy()
    
    # Create a column to store the stats
    stat_col_name = f'{category_col}_target_mean'
    data_copy[stat_col_name] = 0.0
    
    for train_index, val_index in kf.split(data):
        train_data = data.iloc[train_index]
        val_data = data.iloc[val_index]
        
        # Calculate mean target value for each category in the training data
        category_means = train_data.groupby(category_col)[target_col].mean()
        
        # Apply the mean values to the validation data
        data_copy.loc[val_index, stat_col_name] = val_data[category_col].map(category_means)
    data_copy.update(data_copy[stat_col_name].fillna(st.mode(data_copy[stat_col_name])))
    return data_copy,stat_col_name
k = 10
z=["Name", "Cabin", "Ticket"]
for name in z:
    x,y= k_fold_category_stats(df, target_col='Survived', category_col=name, k=k)
    df[name]=x[y]
df["Sex"].loc[df["Sex"]=="male"]=1
df["Sex"].loc[df["Sex"]=="female"]=0
w=list(df["Embarked"].unique())
for a in w:
    n=list(df["Embarked"]==str(a))
    for b in range(len(n)):
        if n[b]==True:
            n[b]=1
        else:
            n[b]=0
    df[a]=n
df.drop(columns="Embarked",inplace=True)
print(df.corr()["Survived"])


C:\Users\Sara\AppData\Local\Temp\ipykernel_14568\2496899170.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(x,inplace=True)


PassengerId    0.073366
Survived       1.000000
Pclass        -0.124679
Name                NaN
Sex           -0.438652
Age            0.017142
SibSp          0.177520
Parch          0.157561
Ticket         0.345441
Fare           0.356754
Cabin          0.178176
S             -0.040517
C              0.168618
Q              0.030105
0                   NaN
Name: Survived, dtype: float64


C:\Users\Sara\AppData\Local\Temp\ipykernel_14568\2496899170.py:90: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Sex"].loc[df["Sex"]=="male"]=1
C:\Users\Sara\AppData\Local\Temp\ipykernel_14568\2496899170.py:90: SettingWithCopyWarning: 
A